In [5]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy import signal
import time
import datetime
import serial
from scipy.fftpack import fft,ifft
from matplotlib.pylab import mpl
import struct
import os
import serial
from pathlib import Path

mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['axes.unicode_minus'] = False

In [3]:
!python.exe -m PyQt5.uic.pyuic Automated_testingV1.6.ui -o Automated_testingV16.py

In [10]:
!python.exe -m PyQt5.uic.pyuic Automated_testingV1.7.ui -o Automated_testingV17.py

In [58]:
!python.exe -m PyQt5.uic.pyuic Automated_testing.ui -o Automated_testing.py

In [4]:
a = '1231254'
print(a[:20])

1231254


In [31]:
def del_empty_folder(path):
    for folder in os.listdir(path):
        folder_path = os.path.join(path,folder)
        if os.path.isdir(folder_path):
            del_empty_folder(folder_path)
            if not os.listdir(folder_path):
                os.rmdir(folder_path)
del_empty_folder('D:\\Data\\BD_Data\\标定数据\\测试\\202503')

In [33]:
str(Path('D:/Data/BD_Data/标定数据/测试/202503'))

'D:\\Data\\BD_Data\\标定数据\\测试\\202503'

In [14]:
test = b'\x55\xaa\xcc\xdd'
test_list = struct.unpack('>BBBB',test)

In [20]:
bit_data = []
bit_data+=test_list
bit_data

[85, 170, 204, 221]

In [4]:
select_plan_name = '01_自动测试'
plan_rule_name = './自动规则/{}.txt'.format(select_plan_name)
plan_files = pd.read_csv(plan_rule_name,sep='\\s+',header=None,encoding='gb2312')

In [24]:
a = [1,2,3]
for j in range(3):
    for i in range(len(a)):
        a[i]+=a[i]
print(a)
for i in range(len(a)):
    a[i] = a[i]/3
print(a)

[8, 16, 24]
[2.6666666666666665, 5.333333333333333, 8.0]


In [34]:
a[2].is_integer()

True

In [35]:
receive_data_s = [2.6666666666666665, 5.333333333333333, 8.0]
save_decimal_point = 6
['{:.{}f}'.format(i,save_decimal_point)if not i.is_integer() else str(int(i)) for i in receive_data_s]

['2.666667', '5.333333', '8']

In [44]:
plot_axis_text = '1  1235ms计数'
try:axis_plot = int(plot_axis_text.split()[0])
except:axis_plot = 0
axis_plot

1

In [1]:
type(0.01)==float

True

In [11]:

strings = '55aaac00000003b900074fd13c76977dbeb616d03f2104e97f1f9bab3d3886c64301861703ac03a603b603b803c200000000000000000500000000000000c0ff'
strings

'55aaac00000003b900074fd13c76977dbeb616d03f2104e97f1f9bab3d3886c64301861703ac03a603b603b803c200000000000000000500000000000000c0ff'

In [12]:
bytes.fromhex(strings)

b'U\xaa\xac\x00\x00\x00\x03\xb9\x00\x07O\xd1<v\x97}\xbe\xb6\x16\xd0?!\x04\xe9\x7f\x1f\x9b\xab=8\x86\xc6C\x01\x86\x17\x03\xac\x03\xa6\x03\xb6\x03\xb8\x03\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\xc0\xff'

In [17]:
split_data = ''
split_list = []
for i in range(len(strings)//2):
    split_data+=strings[i*2:i*2+2]+' '
    split_list.append(strings[i*2:i*2+2])
split_data

'55 aa ac 00 00 00 03 b9 00 07 4f d1 3c 76 97 7d be b6 16 d0 3f 21 04 e9 7f 1f 9b ab 3d 38 86 c6 43 01 86 17 03 ac 03 a6 03 b6 03 b8 03 c2 00 00 00 00 00 00 00 00 05 00 00 00 00 00 00 00 c0 ff '

In [3]:
__dict__['test'] = 'test'

NameError: name '__dict__' is not defined

In [1]:
len('EB 90 41 00 91 03 2F BC 58 F8 44 3B 89 8D 40 BC 08 F7 08 BD 0A 6D 2F 3C A9 65 1C C1 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 13 1F ')//3

68

In [ ]:
import serial
import time

class turnTable3x:
    def __init__(self):
        self.list_debugMsg = []
        self.list_showMsg = []
        self.list_mode = [0, 0, 0]  # 运行模式
        self.list_commandLocation = [0, 0, 0]
        self.list_commandSpeed = [0, 0, 0]
        self.list_commandAcceleration = [10, 10, 10]
        self.flag_theading = False
        self.serial = None
        self.serial_com = 'COM1'
        self.serial_check = False

    def append_debugMsg(self, msg):
        if isinstance(msg, list):
            msg = ', '.join([str(i) for i in msg])
        self.list_debugMsg.append(str(msg))
        while len(self.list_debugMsg) > 10:
            self.list_debugMsg.pop(0)

    def append_showMsg(self, msg):
        if isinstance(msg, list):
            msg = ', '.join([str(i) for i in msg])
        self.list_showMsg.append(str(msg))
        while len(self.list_showMsg) > 100:
            self.list_showMsg.pop(0)
    def serial_open(self):
        if self.serial is None:
            try:
                self.serial = serial.Serial(self.serial_com, 115200)
            except Exception as e:
                self.append_debugMsg('开启串口异常: {}'.format(e))
                self.serial = None
        else:
            try:
                self.serial.close()
                self.serial = None
                self.serial = serial.Serial(self.serial_com, 115200)
            except Exception as e:
                self.append_debugMsg('重启串口异常: {}'.format(e))
                self.serial = None

    def serial_close(self):
        if self.serial is None:
            return True
        else:
            try:
                self.serial.close()
                self.serial = None
            except Exception as e:
                self.append_debugMsg('重启串口异常: {}'.format(e))
    # 尝试读取串口数据
    def serial_tryRec(self, tryCount=10, waitTime=0.02):
        count = 0
        while count < tryCount:
            count += 1
            try:
                wait = self.serial.in_waiting
                if wait > 0:
                    recdata = self.serial.read(wait)
                    return recdata
            except Exception as e:
                self.append_debugMsg('获取数据错误: {}'.format(e))
                time.sleep(waitTime)
        return False
    # 尝试发送和读取校验
    def sendAndRec(self, sendMsg='', recMsg=''):
        if len(sendMsg) > 0:
            # print('sendMsg: {}'.format(sendMsg))
            try:
                self.serial.write(sendMsg.encode())
            except Exception as e:
                self.append_debugMsg('tt_chk send error: {}'.format(e))
                return False
        try:
            rec = self.serial_tryRec()
            if not rec:
                self.append_debugMsg('未接收到转台指令')
                return False
            if len(recMsg) > 0:
                if recMsg in rec.decode():
                    return True
                else:
                    self.append_showMsg('回读校验失败: {}'.format(recMsg))
                    self.append_debugMsg(
                        '回读校验失败: send:<{}> check:<{}> rec:<{}>'.format(sendMsg, recMsg, rec.decode())
                    )
                    return False
            else:
                return rec.decode()
        except Exception as e:
            self.append_debugMsg('tt_chk rec error: {}'.format(e))
            return False
        return False

    # 通讯校验和
    def sumCheckAscii(self, string):
        return sum(ord(c) for c in string) & 0xFF

    # 通讯检查
    def sendMsg_chk(self):
        return self.sendAndRec('$MNCHK,1*CE\r\n', '$ASCHK,OK*30')
    # 进入远控
    def sendMsg_rem(self):
        return self.sendAndRec('$MNREM,1*DC\r\n', '$ASREM,OK*3E')
    # 返回本控
    def sendMsg_loc(self):
        return self.sendAndRec('$MNLOC,1*D6\r\n', '$ASLOC,OK*38')
    # 使能
    def sendMsg_enb(self):
        return self.sendAndRec('$MNENB,1*CD\r\n', '$ASENB,OK*2F')
    # 断开使能
    def sendMsg_dis(self):
        return self.sendAndRec('$MNDIS,1*D8\r\n', '$ASDIS,OK*3A')
    # 转台寻零
    def sendMsg_hmz(self):
        return self.sendAndRec('$MNHMZ,1*E7\r\n', '$ASHMZ,OK*49')
    # 运行模式
    def sendMsg_mod(self, a, b, c):
        msg_list = [a, b, c]
        for i in msg_list:
            if str(i) not in ['0', '1', '2']:
                self.append_showMsg('转台运行模式设置错误: {}'.format([a, b, c]))
                return False
        sendMsg = 'MNMOD,{},{},{}'.format(a, b, c)
        recMsg = 'ASMOD,{},{},{},OK'.format(a, b, c)
        return self.sendAndRec(
            '${}*{:02X}\r\n'.format(sendMsg, self.sumCheckAscii(sendMsg)),
            '${}*{:02X}'.format(recMsg, self.sumCheckAscii(recMsg))
            )
    # 位置设置
    def sendMsg_pos(self, a, b, c):
        msg_list = [a, b, c]
        for i in msg_list:
            try:float(i)
            except:
                self.append_showMsg('转台位置设置错误: {}'.format([a, b, c]))
                return False
        sendMsg = 'MNPOS,{},{},{}'.format(a, b, c)
        recMsg = 'ASPOS,{},{},{},OK'.format(a, b, c)
        return self.sendAndRec(
            '${}*{:02X}\r\n'.format(sendMsg, self.sumCheckAscii(sendMsg)),
            '${}*{:02X}'.format(recMsg, self.sumCheckAscii(recMsg))
            )
    # 速度设置
    def sendMsg_vel(self, a, b, c):
        msg_list = [a, b, c]
        send_list = []
        for i in msg_list:
            try:send_list.append(float(i))
            except:
                self.append_showMsg('转台速度设置错误: {}'.format([a, b, c]))
                return False
        sendMsg = 'MNVEL,{},{},{}'.format(a, b, c)
        recMsg = 'ASVEL,{},{},{},OK'.format(a, b, c)
        return self.sendAndRec(
            '${}*{:02X}\r\n'.format(sendMsg, self.sumCheckAscii(sendMsg)),
            '${}*{:02X}'.format(recMsg, self.sumCheckAscii(recMsg))
            )
    # 加速度设置
    def sendMsg_acc(self, a, b, c):
        msg_list = [a, b, c]
        for i in msg_list:
            try:float(i)
            except:
                self.append_showMsg('转台加速度设置错误: {}'.format([a, b, c]))
                return False
        sendMsg = 'MNACC,{},{},{}'.format(a, b, c)
        recMsg = 'ASACC,{},{},{},OK'.format(a, b, c)
        return self.sendAndRec(
            '${}*{:02X}\r\n'.format(sendMsg, self.sumCheckAscii(sendMsg)),
            '${}*{:02X}'.format(recMsg, self.sumCheckAscii(recMsg))
            )
    # 运行
    def sendMsg_run(self):
        return self.sendAndRec('$MNRUN,1*ED\r\n', '$ASRUN,OK*4F')
    # 停止
    def sendMsg_stp(self):
        return self.sendAndRec('$MNSTP,1*EF\r\n', '$ASSTP,OK*51')
    # 状态查询
    def sendMsg_sts(self):
        return self.sendAndRec('$MNSTS,1*F2\r\n', '')
    def set

In [15]:
test = turnTable3x()
test.serial_open()

In [19]:
test.sendMsg_mod(0,0,0)

sendMsg: $MNMOD,0,0,0*8F



False

In [23]:
test.sendMsg_pos('0.0000',123.4567,345.6789)

sendMsg: $MNPOS,0.0000,123.4567,345.6789*71



False

In [27]:
test.sendMsg_vel(12.3456,-123.4567,'10.0000')

sendMsg: $MNVEL,12.3456,-123.4567,10.0000*7F



False

In [36]:
test.sendMsg_acc('10.0000','20.0000','30.0000')

sendMsg: $MNACC,10.0000,20.0000,30.0000*D6



False

In [35]:
hex(test.sumCheckAscii('MNVEL,12.3456,-123.4567,10.0000'))

'0x7f'

In [34]:
hex(test.sumCheckAscii('ASVEL,12.3456,-123.4567,10.0000,OK'))

'0x3e'

In [12]:
rec = b'$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n$ASSTP,OK*51\r\n'
recMsg = '$ASCHK,OK*30'
recMsg in rec.decode()

False